In [1]:
# Testing the downloading data method from:
# https://claut.gitlab.io/man_ccia/lab2.html
# Import the required modules
from pyesgf.search import SearchConnection
import os
import sys
import importlib
import pandas as pd
import requests
from tqdm import tqdm

# Set the os environment to on
os.environ['ESGF_PYCLIENT_NO_FACETS_STAR_WARNING'] = "on"

In [2]:
# Import the functions
from testing_download_functions import query_data_esgf, extract_file_context, \
                                        download_file, extract_file_context_multithread, \
                                        check_file_exists_jasmin, query_models_esgf

In [3]:
# Import the functions again
# import the functions
_ = importlib.reload(sys.modules['testing_download_functions'])

# Import the functions
from testing_download_functions import query_data_esgf, extract_file_context, \
                                        download_file, extract_file_context_multithread, \
                                        check_file_exists_jasmin, query_models_esgf, find_valid_nodes

In [4]:
# set the search connection
# to the LLNL search node
conn = SearchConnection('https://esgf-node.llnl.gov/esg-search', distrib=True)

In [5]:
# Find the models which have data on the esgf node for the following constraints
# The constraints are:
experiment_id = 'dcppA-hindcast'
latest = True
variable_id = 'rsds'
project = 'CMIP6'
table_id = 'Amon'
activity_id = 'DCPP'
sub_experiment_id = 's1961'

In [6]:
# Test the function which does the same as below
models_list = query_models_esgf(experiment_id=experiment_id,
                                variable_id=variable_id,
                                table_id=table_id,
                                activity_id=activity_id,
                                connection=conn,
                                latest=latest,
                                project=project,
                                sub_experiment_id=sub_experiment_id)

# Print the models list
print(models_list)

Found 473 results.
{'HadGEM3-GC31-MM', 'BCC-CSM2-MR', 'NorCPM1', 'EC-Earth3', 'MRI-ESM2-0', 'MIROC6', 'FGOALS-f3-L', 'CanESM5', 'MPI-ESM1-2-HR', 'CESM1-1-CAM5-CMIP5', 'CMCC-CM2-SR5', 'IPSL-CM6A-LR'}


In [7]:
# Set up the params for the query
params = {
    "latest": latest,
    "project": project,
    "experiment_id": experiment_id,
    "variable_id": variable_id,
    "activity_id": activity_id,
    "table_id": table_id
}

# Query the database
query = conn.new_context(**params)

# Get the results
results = query.search()

print(len(results))

# Python
# Form a list of the unique 'source_id' values from the results
# Python
# Form a set of the unique 'source_id' values from the results
source_id_set = set(id for result in results for id in result.json['source_id'])

# Print the set
print(source_id_set)

27769


KeyboardInterrupt: 

In [15]:
print(type(models_list))

# # Convert the set to a list
source_id_list = list(models_list)

# Constrain to the first 3 members
source_id_list = source_id_list[:3]

source_id_set = source_id_list

<class 'set'>


In [18]:
# Set the params for the query
params = {
    "latest": latest,
    "project": project,
    "experiment_id": experiment_id,
    "variable_id": variable_id,
    "activity_id": activity_id,
    "table_id": table_id,
    "sub_experiment_id": sub_experiment_id
}

In [19]:
# # Constrain the source_id_set to the first 1 model
# source_id_set = list(source_id_set)[0:1]

# # Print the set
# print(source_id_set)

# Initialize an empty dictionary to store the results
max_results = {'source_id': None, 'data_node': None, 'num_results': 0}

# Create a list for the max_results dictionaries
max_results_list = []

# Set up the max results per source dictionary
max_results_per_source = {}

# Loop through the source_id_set and query which nodes have data for each model
for source_id in source_id_set:
    print("trying to find valid nodes for model: {}".format(source_id))
    # Set the source_id constraint
    params['source_id'] = source_id
    print(params)
    # Query the database
    model_query = conn.new_context(**params)
    # Get the results
    model_results = model_query.search()
    # Print the number of results
    print(len(model_results))

    # if the len of the model results is not 0
    if len(model_results) != 0:
        # Print the first result
        print(model_results[0].json['id'])

    # Identify the unique nodes (data_node) which have data for the model
    data_node_set = set(result.json['data_node'] for result in model_results)

    # Print the set
    print(data_node_set)

    # Loop through the data_node_set and query how many files are available for each 
    # node
    for data_node in data_node_set:
        print("trying to find valid files for node: {}".format(data_node))
        
        # Set up the params for the query
        params_node = params.copy()
        
        # Set the data_node constraint
        params_node['data_node'] = data_node
        # Query the database
        node_query = conn.new_context(**params_node)
        # Get the results
        node_results = node_query.search()
        # Print the number of results
        print(len(node_results))

        # If this source_id is not in max_results_per_source or this data_node has more results, update the dictionary
        if source_id not in max_results_per_source or len(node_results) > max_results_per_source[source_id]:
            max_results = {'source_id': source_id, 'data_node': data_node, 'num_results': len(node_results)}
            max_results_per_source[source_id] = len(node_results)

            # Append the max_results dictionary to the list
            max_results_list.append(max_results)
        else:
            print("this data_node has less results than the previous one")
            continue

# Print the dictionary
print(max_results_list)





trying to find valid nodes for model: HadGEM3-GC31-MM
{'latest': True, 'project': 'CMIP6', 'experiment_id': 'dcppA-hindcast', 'variable_id': 'rsds', 'activity_id': 'DCPP', 'table_id': 'Amon', 'sub_experiment_id': 's1961', 'source_id': 'HadGEM3-GC31-MM'}
20
CMIP6.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1961-r3i1p1f2.Amon.rsds.gn.v20200417|esgf-data1.llnl.gov
{'esgf.ceda.ac.uk', 'esgf-data1.llnl.gov'}
trying to find valid files for node: esgf.ceda.ac.uk
10
trying to find valid files for node: esgf-data1.llnl.gov
10
this data_node has less results than the previous one
trying to find valid nodes for model: BCC-CSM2-MR
{'latest': True, 'project': 'CMIP6', 'experiment_id': 'dcppA-hindcast', 'variable_id': 'rsds', 'activity_id': 'DCPP', 'table_id': 'Amon', 'sub_experiment_id': 's1961', 'source_id': 'BCC-CSM2-MR'}
32
CMIP6.DCPP.BCC.BCC-CSM2-MR.dcppA-hindcast.s1961-r1i1p1f1.Amon.rsds.gn.v20191126|cmip.bcc.cma.cn
{'cmip.bcc.cma.cn', 'esgf3.dkrz.de', 'esgf.ceda.ac.uk', 'esgf-data1.llnl.gov'}
t

In [22]:
# Test the function for finding the valid nodes
max_results_list = find_valid_nodes(params=params, 
                                    models_list=models_list,
                                    conn=conn)

trying to find valid nodes for model: IPSL-CM6A-LR
{'latest': True, 'project': 'CMIP6', 'experiment_id': 'hist-GHG', 'variable_id': 'tas', 'activity_id': 'DAMIP', 'table_id': 'Amon', 'source_id': 'IPSL-CM6A-LR'}
51
{'vesg.ipsl.upmc.fr', 'esgf.ceda.ac.uk', 'esgf3.dkrz.de', 'aims3.llnl.gov', 'esgf.nci.org.au', 'esgf-data04.diasjp.net'}
trying to find valid files for node: vesg.ipsl.upmc.fr
10
trying to find valid files for node: esgf.ceda.ac.uk
10
trying to find valid files for node: esgf3.dkrz.de
10
trying to find valid files for node: aims3.llnl.gov
10
trying to find valid files for node: esgf.nci.org.au
10
trying to find valid files for node: esgf-data04.diasjp.net
1
trying to find valid nodes for model: GFDL-ESM4
{'latest': True, 'project': 'CMIP6', 'experiment_id': 'hist-GHG', 'variable_id': 'tas', 'activity_id': 'DAMIP', 'table_id': 'Amon', 'source_id': 'GFDL-ESM4'}
6
{'esgf.ceda.ac.uk', 'esgf3.dkrz.de', 'esgf-data1.llnl.gov', 'esgf.nci.org.au', 'esgf-data04.diasjp.net', 'esgdata.g

In [20]:
# Clean the max_results_list to remove duplicate source_id entries
# Keep the entry with the highest number of results (num_results)
# Initialize an empty list to store the unique source_id entries
unique_source_id_list = []

# Loop through the max_results_list and append the unique source_id entries
for result in max_results_list:
    if result['source_id'] not in unique_source_id_list:
        unique_source_id_list.append(result['source_id'])

# Print the list
print(unique_source_id_list)

# Initialize an empty list to store the unique max_results_list entries
unique_max_results_list = []

# Loop through the unique_source_id_list and only
# Append the max_results_list entries which match the source_id and have the highest num_results
for source_id in unique_source_id_list:
    print("source_id: {}".format(source_id))
    # Initialize an empty list to store the num_results
    num_results_list = []
    # Loop through the max_results_list and append the num_results to the list
    for result in max_results_list:
        if result['source_id'] == source_id:
            num_results_list.append(result['num_results'])
    # Get the max num_results
    max_num_results = max(num_results_list)
    # Loop through the max_results_list and append the entries which match the source_id and max_num_results
    for result in max_results_list:
        if result['source_id'] == source_id and result['num_results'] == max_num_results:
            unique_max_results_list.append(result)

['HadGEM3-GC31-MM', 'BCC-CSM2-MR', 'NorCPM1']
source_id: HadGEM3-GC31-MM
source_id: BCC-CSM2-MR
source_id: NorCPM1


In [9]:
# Convert the unique_max_results_list to a dataframe
unique_max_results_df = pd.DataFrame.from_dict(unique_max_results_list)

# Print the dataframe
unique_max_results_df

# save the dataframe
# save in current directory + save_data + filename
save_dir = os.path.join(os.getcwd(), 'save_data')
save_filename = 'unique_max_results_df_{}_{}_{}_{}_{}.csv'.format(experiment_id, variable_id, project, table_id, activity_id)

# Form the save path
save_path = os.path.join(save_dir, save_filename)

# Check if the save directory exists
if not os.path.exists(save_dir):
    # Make the directory
    os.makedirs(save_dir)

# Save the dataframe
unique_max_results_df.to_csv(save_path)

In [7]:
save_dir = os.path.join(os.getcwd(), 'save_data')
save_filename = 'unique_max_results_df_{}_{}_{}_{}_{}.csv'.format(experiment_id, variable_id, project, table_id, activity_id)

# Form the save path
save_path = os.path.join(save_dir, save_filename)

In [24]:
print(type(unique_max_results_list))

print(unique_max_results_list)

# Convert the unique_max_results_list to a dataframe
unique_max_results_df = pd.DataFrame.from_dict(unique_max_results_list)

# Print the dataframe
unique_max_results_df

<class 'list'>
[{'source_id': 'HadGEM3-GC31-MM', 'data_node': 'esgf.ceda.ac.uk', 'num_results': 10}, {'source_id': 'BCC-CSM2-MR', 'data_node': 'cmip.bcc.cma.cn', 'num_results': 8}, {'source_id': 'NorCPM1', 'data_node': 'noresg.nird.sigma2.no', 'num_results': 20}]


,source_id,data_node,num_results
0,HadGEM3-GC31-MM,esgf.ceda.ac.uk,10
1,BCC-CSM2-MR,cmip.bcc.cma.cn,8
2,NorCPM1,noresg.nird.sigma2.no,20


In [26]:
# Open the save_path as a dataframe
# unique_max_results_df = pd.read_csv(save_path)

# # Convert unique_max_results to a dataframe
# unique_max_results_df = pd.DataFrame.from_dict(unique_max_results_list)

results_list = []

# Loop over the dataframe to create a list of result sets
for i in range(len(unique_max_results_df)):
    
    # Get the source_id and data_node
    source_id = unique_max_results_df.loc[i, 'source_id']
    data_node = unique_max_results_df.loc[i, 'data_node']

    # Print the source_id and data_node
    print("source_id: {}, data_node: {}".format(source_id, data_node))

    # Print the experiment_id, variable_id, activity_id, and data_node
    print("experiment_id: {}, variable_id: {}, activity_id: {}, data_node: {}".format(experiment_id, variable_id, activity_id, data_node))

    results = query_data_esgf(conn,
                            source_id = source_id,
                            experiment_id = experiment_id,
                            variable_id = variable_id,
                            table_id = table_id,
                            project=project,
                            activity_id = activity_id,
                            data_node = data_node,
                            sub_experiment_id=sub_experiment_id
    )

    print(len(results))

    # Append the results to the results_list
    results_list.append(results)

# Print the results_list
print(results_list)

source_id: HadGEM3-GC31-MM, data_node: esgf.ceda.ac.uk
experiment_id: dcppA-hindcast, variable_id: rsds, activity_id: DCPP, data_node: esgf.ceda.ac.uk
10
source_id: BCC-CSM2-MR, data_node: cmip.bcc.cma.cn
experiment_id: dcppA-hindcast, variable_id: rsds, activity_id: DCPP, data_node: cmip.bcc.cma.cn
8
source_id: NorCPM1, data_node: noresg.nird.sigma2.no
experiment_id: dcppA-hindcast, variable_id: rsds, activity_id: DCPP, data_node: noresg.nird.sigma2.no
20
[<pyesgf.search.results.ResultSet object at 0x7f67ae021420>, <pyesgf.search.results.ResultSet object at 0x7f683466f400>, <pyesgf.search.results.ResultSet object at 0x7f67ad5205b0>]


In [33]:
# Set up a list for the file context
file_context_list = []
failed_results_list = []

# Constrain results_list to the first 3 results
results_list = results_list[0:3]

# Loop through the results_list and print ther number of results
for results in results_list:
    print(len(results))

    # Extract the file context from the results
    file_context, failed_results = extract_file_context(results)

    # Append the file_context to the file_context_list
    file_context_list.append(file_context)

    # Append the failed_results to the failed_results_list
    failed_results_list.append(failed_results)


# Print the file_context_list
print(file_context_list)

# Print the failed_results_list
print(failed_results_list)

10
Extracting file context for 10 datasets...


 10%|█         | 1/10 [00:08<01:18,  8.75s/it]

Processed 1 out of 10 results.


 20%|██        | 2/10 [00:17<01:09,  8.68s/it]

Processed 2 out of 10 results.


...

Processed 18 out of 20 results.


 95%|█████████▌| 19/20 [03:17<00:10, 10.45s/it]

Processed 19 out of 20 results.


100%|██████████| 20/20 [03:28<00:00, 10.40s/it]

Processed 20 out of 20 results.
[[{'filename': 'rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196111-196112.nc', 'url': 'https://esgf.ceda.ac.uk/thredds/fileServer/esg_cmip6/CMIP6/DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1961-r10i1p1f2/Amon/rsds/gn/v20200417/rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196111-196112.nc'}, {'filename': 'rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196201-196212.nc', 'url': 'https://esgf.ceda.ac.uk/thredds/fileServer/esg_cmip6/CMIP6/DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1961-r10i1p1f2/Amon/rsds/gn/v20200417/rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196201-196212.nc'}, {'filename': 'rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196301-196312.nc', 'url': 'https://esgf.ceda.ac.uk/thredds/fileServer/esg_cmip6/CMIP6/DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1961-r10i1p1f2/Amon/rsds/gn/v20200417/rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196301-196312.nc'}, {'file

In [34]:
print(len(file_context_list))
print(type(file_context_list))
print(file_context_list)

for file_context in file_context_list:
    print(type(file_context))
    print(file_context)

3
<class 'list'>
[[{'filename': 'rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196111-196112.nc', 'url': 'https://esgf.ceda.ac.uk/thredds/fileServer/esg_cmip6/CMIP6/DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1961-r10i1p1f2/Amon/rsds/gn/v20200417/rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196111-196112.nc'}, {'filename': 'rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196201-196212.nc', 'url': 'https://esgf.ceda.ac.uk/thredds/fileServer/esg_cmip6/CMIP6/DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1961-r10i1p1f2/Amon/rsds/gn/v20200417/rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196201-196212.nc'}, {'filename': 'rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196301-196312.nc', 'url': 'https://esgf.ceda.ac.uk/thredds/fileServer/esg_cmip6/CMIP6/DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1961-r10i1p1f2/Amon/rsds/gn/v20200417/rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196301-196312.nc'}, {'filename': 'rsds_Am

In [35]:
# Create an empty DataFrame
df = pd.DataFrame()

# Loop through the file_context_list
for file_context in file_context_list:
    # Convert the dictionary to a DataFrame
    temp_df = pd.DataFrame(file_context)

    # Append the temp_df to the main df
    df = pd.concat([df, temp_df], ignore_index=True)

# Print the df
df

,filename,url
0,rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961...,https://esgf.ceda.ac.uk/thredds/fileServer/esg...
1,rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961...,https://esgf.ceda.ac.uk/thredds/fileServer/esg...
2,rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961...,https://esgf.ceda.ac.uk/thredds/fileServer/esg...
3,rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961...,https://esgf.ceda.ac.uk/thredds/fileServer/esg...
4,rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961...,https://esgf.ceda.ac.uk/thredds/fileServer/esg...
...,...,...
143,rsds_Amon_NorCPM1_dcppA-hindcast_s1961-r10i2p1...,http://noresg.nird.sigma2.no/thredds/fileServe...
144,rsds_Amon_NorCPM1_dcppA-hindcast_s1961-r4i2p1f...,http://noresg.nird.sigma2.no/thredds/fileServe...
145,rsds_Amon_NorCPM1_dcppA-hindcast_s1961-r5i1p1f...,http://noresg.nird.sigma2.no/thredds/fileServe...
146,rsds_Amon_NorCPM1_dcppA-hindcast_s1961-r2i2p1f...,http://noresg.nird.sigma2.no/thredds/fileServe...


In [ ]:
# Python
# Create an empty DataFrame
df = pd.DataFrame()

# Loop through the file_context_list
for file_context in file_context_list:
    # Convert the dictionary to a DataFrame
    temp_df = pd.DataFrame.from_dict(file_context)

    # Append the temp_df to the main df
    df = pd.concat([df, temp_df], ignore_index=True)

In [40]:
# reload for modules
import importlib
import sys

# import the functions
_ = importlib.reload(sys.modules['testing_download_functions'])

from testing_download_functions import query_data_esgf, extract_file_context, \
                                        download_file, extract_file_context_multithread, \
                                        check_file_exists_jasmin

In [41]:
# Check whether the files exist on JASMIN
import glob

# Set up the file_context_df
file_context_df = df.copy()

# We want to verify whether these files exist on JASMIN
damip_dir = "/badc/cmip6/data/CMIP6/DCPP/"

# Test the function
# TODO
files_df = check_file_exists_jasmin(file_context_df, damip_dir)

files_df

MOHC
badc
File exists for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196111-196112.nc
MOHC
badc
File exists for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196201-196212.nc
MOHC
badc
File exists for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196301-196312.nc
MOHC
badc
File exists for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196401-196412.nc
MOHC
badc
File exists for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196501-196512.nc
MOHC
badc
File exists for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196601-196612.nc
MOHC
badc
File exists for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196701-196712.nc
MOHC
badc
File exists for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196801-196812.nc
MOHC
badc
File exists for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196901-196912.nc
MOHC
badc
File exists for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-

,filename,url,file_exists,filepath
0,rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961...,https://esgf.ceda.ac.uk/thredds/fileServer/esg...,True,/badc/cmip6/data/CMIP6/DCPP/MOHC/HadGEM3-GC31-...
1,rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961...,https://esgf.ceda.ac.uk/thredds/fileServer/esg...,True,/badc/cmip6/data/CMIP6/DCPP/MOHC/HadGEM3-GC31-...
2,rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961...,https://esgf.ceda.ac.uk/thredds/fileServer/esg...,True,/badc/cmip6/data/CMIP6/DCPP/MOHC/HadGEM3-GC31-...
3,rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961...,https://esgf.ceda.ac.uk/thredds/fileServer/esg...,True,/badc/cmip6/data/CMIP6/DCPP/MOHC/HadGEM3-GC31-...
4,rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961...,https://esgf.ceda.ac.uk/thredds/fileServer/esg...,True,/badc/cmip6/data/CMIP6/DCPP/MOHC/HadGEM3-GC31-...
...,...,...,...,...
143,rsds_Amon_NorCPM1_dcppA-hindcast_s1961-r10i2p1...,http://noresg.nird.sigma2.no/thredds/fileServe...,True,/badc/cmip6/data/CMIP6/DCPP/NCC/NorCPM1/dcppA-...
144,rsds_Amon_NorCPM1_dcppA-hindcast_s1961-r4i2p1f...,http://noresg.nird.sigma2.no/thredds/fileServe...,True,/badc/cmip6/data/CMIP6/DCPP/NCC/NorCPM1/dcppA-...
145,rsds_Amon_NorCPM1_dcppA-hindcast_s1961-r5i1p1f...,http://noresg.nird.sigma2.no/thredds/fileServe...,True,/badc/cmip6/data/CMIP6/DCPP/NCC/NorCPM1/dcppA-...
146,rsds_Amon_NorCPM1_dcppA-hindcast_s1961-r2i2p1f...,http://noresg.nird.sigma2.no/thredds/fileServe...,True,/badc/cmip6/data/CMIP6/DCPP/NCC/NorCPM1/dcppA-...


In [42]:
# We only want to download the files which don't already exist on JASMIN
dcpp_dir_gws = "/gws/nopw/j04/canari/users/benhutch/"


# Now we want to check whether the files exist on JASMIN
files_df = check_file_exists_jasmin(files_df, dcpp_dir_gws)

MOHC
gws
File does not exist for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196111-196112.nc
MOHC
gws
File does not exist for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196201-196212.nc
MOHC
gws
File does not exist for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196301-196312.nc
MOHC
gws
File does not exist for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196401-196412.nc
MOHC
gws
File does not exist for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196501-196512.nc
MOHC
gws
File does not exist for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196601-196612.nc
MOHC
gws
File does not exist for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196701-196712.nc
MOHC
gws
File does not exist for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196801-196812.nc
MOHC
gws
File does not exist for rsds_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1961-r10i1p1f2_gn_196901-196912.nc
MOHC
gws
F

In [23]:
# We only want to download the files which don't already exist on JASMIN
# Set up the download directory
download_dir = "/gws/nopw/j04/scenario/users/benhutch/"

# Loop through files_df and download the files
# First constrain the dataframe to only the files which don't already exist on JASMIN
files_df = files_df[files_df['file_exists'] == False]

# Reset the index
files_df.reset_index(drop=True, inplace=True)

# Loop through the files_df and download the files
for i in tqdm(range(len(files_df))):
    # Get the file_url
    file_url = files_df.loc[i, 'url']

    # Get the filename
    filename = files_df.loc[i, 'filename']

    # Split the filename and extract the variable name
    variable = filename.split('_')[0]

    # Split the filename to get the experiment name
    experiment = filename.split('_')[3]

    # Set up the model
    model = filename.split('_')[2]

    # Set up the download directory
    download_dir_loop = os.path.join(download_dir, experiment, variable, model)

    # If the download directory doesn't exist, make it
    if not os.path.exists(download_dir_loop):
        os.makedirs(download_dir_loop)

    # Set up the download path
    download_path = os.path.join(download_dir_loop, filename)

    # In the filepath column of the dataframe
    # replace the current file path with the download path
    files_df.loc[i, 'filepath'] = download_path

    # Set up the request
    r = requests.get(file_url, stream=True)

    # Set up the total size
    total_size = int(r.headers.get('content-length', 0))
    
    # Set up the block size
    block_size = 1024

    # Download the file
    with open(download_path, 'wb') as f:
        for data in tqdm(r.iter_content(block_size), 
                        total = total_size//block_size, 
                        unit = 'KiB', 
                        unit_scale = True):
            f.write(data)

        # If the total size is no 0
        if total_size != 0:
            print("File is not empty")
            print("Download complete - file saved to {}".format(download_path))


  0%|          | 0/266 [00:00<?, ?it/s]

145kKiB [00:16, 8.91kKiB/s]                         


File is not empty
Download complete - file saved to /gws/nopw/j04/scenario/users/benhutch/hist-GHG/tas/CNRM-CM6-1/tas_Amon_CNRM-CM6-1_hist-GHG_r3i1p1f2_gr_185001-202012.nc


228kKiB [01:26, 2.62kKiB/s]                       
  1%|          | 2/266 [01:48<4:25:58, 60.45s/it]

File is not empty
Download complete - file saved to /gws/nopw/j04/scenario/users/benhutch/hist-GHG/tas/MRI-ESM2-0/tas_Amon_MRI-ESM2-0_hist-GHG_r4i1p1f1_gn_185001-202012.nc


...

File is not empty
Download complete - file saved to /gws/nopw/j04/scenario/users/benhutch/hist-GHG/tas/FGOALS-g3/tas_Amon_FGOALS-g3_hist-GHG_r2i1p1f1_gn_193001-193912.nc


6.77kKiB [01:17, 87.8KiB/s]                          
 24%|██▍       | 65/266 [21:25<3:41:20, 66.07s/it]

File is not empty
Download complete - file saved to /gws/nopw/j04/scenario/users/benhutch/hist-GHG/tas/FGOALS-g3/tas_Amon_FGOALS-g3_hist-GHG_r2i1p1f1_gn_198001-198912.nc


 70%|███████   | 4.75k/6.77k [01:10<00:29, 67.4KiB/s]
 24%|██▍       | 65/266 [22:36<1:09:54, 20.87s/it]


KeyboardInterrupt: 

In [14]:
# Extract all of the file_context into dictionaries
# and then append them to a dataframe
# Initialize an empty list to store the dictionaries
# Create an empty dataframe
file_context_df = pd.DataFrame()


# For each file_context
for file_context in file_context_list:
    # Append the file_context to the dataframe
    file_context_df = file_context_df.append(file_context, ignore_index=True)

/tmp/ipykernel_1622/1028569091.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file_context_df = file_context_df.append(file_context, ignore_index=True)
/tmp/ipykernel_1622/1028569091.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file_context_df = file_context_df.append(file_context, ignore_index=True)
/tmp/ipykernel_1622/1028569091.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file_context_df = file_context_df.append(file_context, ignore_index=True)
/tmp/ipykernel_1622/1028569091.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file_context_df = file_context_df.append(file_context, ignore_index=True)
/tmp/ipykern

In [ ]:
# Python
# Form a list of the unique 'source_id' values from the results
source_id_list = list(set([result.json['source_id'] for result in results]))

# Print the list
print(source_id_list)

In [5]:
# test the function for querying the database
results = query_data_esgf(conn,
                        source_id='E3SM-2-0',
                        experiment_id='hist-aer',
                        variable_id='tas',
                        table_id='Amon',
                        data_node='esgf-data2.llnl.gov',)

# print the len of the results
print(len(results))

# print the type of the results
print(type(results))

# print the results
print(results)

5
<class 'pyesgf.search.results.ResultSet'>


In [6]:
# Print the details of the first result
print(results[0].json['id'])

CMIP6.DAMIP.E3SM-Project.E3SM-2-0.hist-aer.r2i1p1f1.Amon.tas.gr.v20220906|esgf-data2.llnl.gov


In [7]:
# Extract the file context
# files_list = extract_file_context(results)

# # # Turn the list into a dataframe
# # files_df = pd.DataFrame.from_dict(files_list)

# # files_df

In [8]:
files_list_mt = extract_file_context_multithread(results)

files_list_mt

Extracting file context for 5 datasets...


[{'filename': 'tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_185001-189912.nc',
  'url': 'https://esgf-data2.llnl.gov/thredds/fileServer/user_pub_work/CMIP6/DAMIP/E3SM-Project/E3SM-2-0/hist-aer/r2i1p1f1/Amon/tas/gr/v20220906/tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_185001-189912.nc'},
 {'filename': 'tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_190001-194912.nc',
  'url': 'https://esgf-data2.llnl.gov/thredds/fileServer/user_pub_work/CMIP6/DAMIP/E3SM-Project/E3SM-2-0/hist-aer/r2i1p1f1/Amon/tas/gr/v20220906/tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_190001-194912.nc'},
 {'filename': 'tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_195001-199912.nc',
  'url': 'https://esgf-data2.llnl.gov/thredds/fileServer/user_pub_work/CMIP6/DAMIP/E3SM-Project/E3SM-2-0/hist-aer/r2i1p1f1/Amon/tas/gr/v20220906/tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_195001-199912.nc'},
 {'filename': 'tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_200001-201412.nc',
  'url': 'https://esgf-data2.llnl.gov/thredds/fileServer/user_pub_work/CMIP6/DAMIP/E3SM-Project/

In [9]:
files_list = files_list_mt

In [11]:
# Print the type of the files list
print(type(files_list))

# Extract this into a dataframe
files_df = pd.DataFrame.from_dict(files_list)
files_df

# Assert that all filenames contrain the string "185001" and "202012"
# assert all(files_df['filename'].str.contains('185001')), "Not all filenames contain the string 185001"
# assert all(files_df['filename'].str.contains('202012')), "Not all filenames contain the string 202012"

files_df

<class 'list'>


,filename,url
0,tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_185001-...,https://esgf-data2.llnl.gov/thredds/fileServer...
1,tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_190001-...,https://esgf-data2.llnl.gov/thredds/fileServer...
2,tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_195001-...,https://esgf-data2.llnl.gov/thredds/fileServer...
3,tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_200001-...,https://esgf-data2.llnl.gov/thredds/fileServer...
4,tas_Amon_E3SM-2-0_hist-aer_r1i1p1f1_gr_185001-...,https://esgf-data2.llnl.gov/thredds/fileServer...
5,tas_Amon_E3SM-2-0_hist-aer_r1i1p1f1_gr_190001-...,https://esgf-data2.llnl.gov/thredds/fileServer...
6,tas_Amon_E3SM-2-0_hist-aer_r1i1p1f1_gr_195001-...,https://esgf-data2.llnl.gov/thredds/fileServer...
7,tas_Amon_E3SM-2-0_hist-aer_r1i1p1f1_gr_200001-...,https://esgf-data2.llnl.gov/thredds/fileServer...
8,tas_Amon_E3SM-2-0_hist-aer_r3i1p1f1_gr_185001-...,https://esgf-data2.llnl.gov/thredds/fileServer...
9,tas_Amon_E3SM-2-0_hist-aer_r3i1p1f1_gr_190001-...,https://esgf-data2.llnl.gov/thredds/fileServer...


In [12]:
import glob

# We want to verify whether these files exist on JASMIN
damip_dir = "/badc/cmip6/data/CMIP6/DAMIP/"

# Test the function
files_df = check_file_exists_jasmin(files_df, damip_dir)

files_df

E3SM-Project
File does not exist for tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_185001-189912.nc
E3SM-Project
File does not exist for tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_190001-194912.nc
E3SM-Project
File does not exist for tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_195001-199912.nc
E3SM-Project
File does not exist for tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_200001-201412.nc
E3SM-Project
File does not exist for tas_Amon_E3SM-2-0_hist-aer_r1i1p1f1_gr_185001-189912.nc
E3SM-Project
File does not exist for tas_Amon_E3SM-2-0_hist-aer_r1i1p1f1_gr_190001-194912.nc
E3SM-Project
File does not exist for tas_Amon_E3SM-2-0_hist-aer_r1i1p1f1_gr_195001-199912.nc
E3SM-Project
File does not exist for tas_Amon_E3SM-2-0_hist-aer_r1i1p1f1_gr_200001-201412.nc
E3SM-Project
File does not exist for tas_Amon_E3SM-2-0_hist-aer_r3i1p1f1_gr_185001-189912.nc
E3SM-Project
File does not exist for tas_Amon_E3SM-2-0_hist-aer_r3i1p1f1_gr_190001-194912.nc
E3SM-Project
File does not exist for tas_Amon_E3SM-2-0_hist-aer_r3i1p1

,filename,url,file_exists
0,tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_185001-...,https://esgf-data2.llnl.gov/thredds/fileServer...,False
1,tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_190001-...,https://esgf-data2.llnl.gov/thredds/fileServer...,False
2,tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_195001-...,https://esgf-data2.llnl.gov/thredds/fileServer...,False
3,tas_Amon_E3SM-2-0_hist-aer_r2i1p1f1_gr_200001-...,https://esgf-data2.llnl.gov/thredds/fileServer...,False
4,tas_Amon_E3SM-2-0_hist-aer_r1i1p1f1_gr_185001-...,https://esgf-data2.llnl.gov/thredds/fileServer...,False
5,tas_Amon_E3SM-2-0_hist-aer_r1i1p1f1_gr_190001-...,https://esgf-data2.llnl.gov/thredds/fileServer...,False
6,tas_Amon_E3SM-2-0_hist-aer_r1i1p1f1_gr_195001-...,https://esgf-data2.llnl.gov/thredds/fileServer...,False
7,tas_Amon_E3SM-2-0_hist-aer_r1i1p1f1_gr_200001-...,https://esgf-data2.llnl.gov/thredds/fileServer...,False
8,tas_Amon_E3SM-2-0_hist-aer_r3i1p1f1_gr_185001-...,https://esgf-data2.llnl.gov/thredds/fileServer...,False
9,tas_Amon_E3SM-2-0_hist-aer_r3i1p1f1_gr_190001-...,https://esgf-data2.llnl.gov/thredds/fileServer...,False


In [10]:
# Set up the directory to download to
download_dir = "/gws/nopw/j04/scenario/users/benhutch/DAMIP"

# Set up the variable
variable = 'tas'

# Set up the experiment id
experiment_id = 'hist-aer'

# Set up the model
model = 'CanESM5'

# Set up the directory
download_path = os.path.join(download_dir, experiment_id, 
                             variable, model)

# Print the download path
print(download_path)

# Use the download function to download a single file
download_file(files_df['url'][0], 
              files_df['filename'][0], download_path)

/gws/nopw/j04/scenario/users/benhutch/DAMIP/hist-aer/tas/CanESM5
Saving to /gws/nopw/j04/scenario/users/benhutch/DAMIP/hist-aer/tas/CanESM5tas_Amon_CanESM5_hist-aer_r1i1p1f1_gn_185001-202012.nc


53.1kKiB [00:46, 1.13kKiB/s]                          


Downloaded size does not match expected size!
 FYI, the status code was  200


In [11]:
# Download all the files
for i in tqdm(range(len(files_df))):
    download_file(files_df['url'][i], 
                  files_df['filename'][i], download_path)

  0%|          | 0/5 [00:00<?, ?it/s]

Saving to /gws/nopw/j04/scenario/users/benhutch/DAMIP/hist-aer/tas/CanESM5tas_Amon_CanESM5_hist-aer_r1i1p1f1_gn_185001-202012.nc


53.1kKiB [00:05, 10.5kKiB/s]                          
 20%|██        | 1/5 [00:05<00:23,  5.86s/it]

Downloaded size does not match expected size!
 FYI, the status code was  200
Saving to /gws/nopw/j04/scenario/users/benhutch/DAMIP/hist-aer/tas/CanESM5tas_Amon_CanESM5_hist-aer_r4i1p1f1_gn_185001-202012.nc


53.1kKiB [00:02, 22.6kKiB/s]                          
 40%|████      | 2/5 [00:08<00:12,  4.25s/it]

...

Saving to /gws/nopw/j04/scenario/users/benhutch/DAMIP/hist-aer/tas/CanESM5tas_Amon_CanESM5_hist-aer_r5i1p1f1_gn_185001-202012.nc


53.1kKiB [00:02, 23.0kKiB/s]                          
 80%|████████  | 4/5 [00:32<00:08,  8.18s/it]

Downloaded size does not match expected size!
 FYI, the status code was  200
Saving to /gws/nopw/j04/scenario/users/benhutch/DAMIP/hist-aer/tas/CanESM5tas_Amon_CanESM5_hist-aer_r3i1p1f1_gn_185001-202012.nc


53.1kKiB [00:13, 3.82kKiB/s]                          
100%|██████████| 5/5 [00:46<00:00,  9.39s/it]

Downloaded size does not match expected size!
 FYI, the status code was  200
